# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bredasdorp,-34.5322,20.0403,62.13,65,59,21.65,ZA,11/26/2024
1,1,George Town,5.4112,100.3354,78.73,88,40,5.99,MY,11/26/2024
2,2,Kapuskasing,49.4169,-82.4331,30.29,100,100,4.61,CA,11/26/2024
3,3,Port Elizabeth,-33.9180,25.5701,68.29,75,0,13.80,ZA,11/26/2024
4,4,Juneau,58.3019,-134.4197,32.00,95,100,8.05,US,11/26/2024


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 1,
    tiles = "EsriNatGeo",
    frame_width = 1500,
    frame_height = 800
)

# Display the map
plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp = (city_data_df["Max Temp"] > 19) & (city_data_df["Max Temp"] < 24)
ideal_humidity = city_data_df.Humidity < 80
ideal_cloud = city_data_df.Cloudiness < 50

# Create Weather DataFrame
ideal_weather_df = city_data_df.loc[ideal_temp & ideal_humidity & ideal_cloud]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
131,131,Mandalgovi,45.7625,106.2708,21.47,51,6,27.09,MN,11/26/2024
185,185,Chippewa Falls,44.9369,-91.3929,23.41,75,20,13.80,US,11/26/2024
266,266,Kalkaman,51.9525,76.0211,19.33,79,22,10.87,KZ,11/26/2024
313,313,Shitanjing,39.2342,106.3439,22.05,39,1,19.86,CN,11/26/2024
451,451,Dalandzadgad,43.5708,104.4250,22.08,50,0,18.59,MN,11/26/2024
569,569,Zhangye,38.9342,100.4517,19.72,63,0,5.32,CN,11/26/2024


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
131,Mandalgovi,MN,45.7625,106.2708,51,
185,Chippewa Falls,US,44.9369,-91.3929,75,
266,Kalkaman,KZ,51.9525,76.0211,79,
313,Shitanjing,CN,39.2342,106.3439,39,
451,Dalandzadgad,MN,43.5708,104.4250,50,
569,Zhangye,CN,38.9342,100.4517,63,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":25
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Mandalgovi - nearest hotel: Б
Chippewa Falls - nearest hotel: Cobblestone Hotel & Suites
Kalkaman - nearest hotel: No hotel found
Shitanjing - nearest hotel: No hotel found
Dalandzadgad - nearest hotel: Хан Уул буудал
Zhangye - nearest hotel: Zhangye Hotel 张掖饭店


,City,Country,Lat,Lng,Humidity,Hotel Name
131,Mandalgovi,MN,45.7625,106.2708,51,Б
185,Chippewa Falls,US,44.9369,-91.3929,75,Cobblestone Hotel & Suites
266,Kalkaman,KZ,51.9525,76.0211,79,No hotel found
313,Shitanjing,CN,39.2342,106.3439,39,No hotel found
451,Dalandzadgad,MN,43.5708,104.4250,50,Хан Уул буудал
569,Zhangye,CN,38.9342,100.4517,63,Zhangye Hotel 张掖饭店


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 1,
    tiles = "EsriNatGeo",
    frame_width = 1500,
    frame_height = 800,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)